Download GNPS data
Replace Job ID below with your GNPS job ID:

In [1]:
! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=a9dc71d3ed5946c795a324b2a1ab4010&view=download_cytoscape_data' -o GNPS_output_graphML.zip
! unzip -d GNPS_output_graphML/ GNPS_output_graphML.zip


Archive:  GNPS_output_graphML.zip
  inflating: GNPS_output_graphML/MOLECULAR-LIBRARYSEARCH-GC-a9dc71d3-download_cytoscape_data-main.graphml  
  inflating: GNPS_output_graphML/params.xml  
  inflating: GNPS_output_graphML/gnps_molecular_network_graphml/6434e3833e1e4376a34ef93b21e39511.graphml  
  inflating: GNPS_output_graphML/Kovats_Result_Filtered_mshub/fc122215454f40ac9632ebe312dc3c05.tsv  
  inflating: GNPS_output_graphML/Kovats_Result_Nonfiltered_mshub/4216fd597908459ea7036a1c4ca2bfab.tsv  


## Run MolNetEnhancer workflow on GCMS data

In [1]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = '/Applications/MarvinSuite/bin/' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [2]:
inchisfile = 'GNPS_output_graphML/Kovats_Result_Filtered_mshub/' + str(os.listdir('GNPS_output_graphML/Kovats_Result_Filtered_mshub/')[0])

In [3]:
inchis = pd.read_csv(inchisfile, sep = "\t", index_col = 0)

In [4]:
inchis = inchis.rename(columns = {'#Scan#':'Scan'})

In [5]:
inchis['SCAN_Mol'] = inchis['Scan'].astype(str) + '_' +inchis['INCHI']

In [6]:
inchis.head(6)

,Scan,Adduct,CAS_Number,Charge,Compound_Name,Compound_Source,Data_Collector,ExactMass,INCHI,INCHI_AUX,...,SpectrumID,TIC_Query,UpdateWorkflowName,tags,Kovats_Index_calculated,Kovats_Index_Lib_Record,Kovats_Index_Error,Kovats_Confidence,Balance_score(percentage),SCAN_Mol
0,1044,EI,75-87-6,1,Chloral,Isolated,NIST,0.0,"InChI=1S/C2HCl3O/c3-2(4,5)1-6/h1H",None,...,CCMSLIB00000934527,273390.0,UPDATE-SINGLE-ANNOTATED-BRONZE,,660.698191,700.280000,0.056523,527.0,NaN,"1044_InChI=1S/C2HCl3O/c3-2(4,5)1-6/h1H"
1,1045,EI,462-06-6,1,"Benzene, fluoro-",Isolated,NIST,0.0,InChI=1S/C6H5F/c7-6-4-2-1-3-5-6/h1-5H,None,...,CCMSLIB00000916220,146657.0,UPDATE-SINGLE-ANNOTATED-BRONZE,,660.698191,664.200000,0.005272,527.0,NaN,1045_InChI=1S/C6H5F/c7-6-4-2-1-3-5-6/h1-5H
2,1047,EI,110-58-7,1,1-Pentanamine,Isolated,NIST,0.0,"InChI=1S/C5H13N/c1-2-3-4-5-6/h2-6H2,1H3",None,...,CCMSLIB00000984202,164741.0,UPDATE-SINGLE-ANNOTATED-BRONZE,,660.698191,731.500000,0.096790,527.0,NaN,"1047_InChI=1S/C5H13N/c1-2-3-4-5-6/h2-6H2,1H3"
3,1047,EI,109-73-9,1,1-Butanamine,Isolated,NIST,0.0,"InChI=1S/C4H11N/c1-2-3-4-5/h2-5H2,1H3",None,...,CCMSLIB00000984143,164741.0,UPDATE-SINGLE-ANNOTATED-BRONZE,,660.698191,616.833333,0.071113,527.0,NaN,"1047_InChI=1S/C4H11N/c1-2-3-4-5/h2-5H2,1H3"
4,1049,EI,108-24-7,1,Acetic anhydride,Isolated,NIST,0.0,InChI=1S/C4H6O3/c1-3(5)7-4(2)6/h1-2H3,None,...,CCMSLIB00000979472,156781.0,UPDATE-SINGLE-ANNOTATED-BRONZE,,659.727553,705.650000,0.065078,1788.0,NaN,1049_InChI=1S/C4H6O3/c1-3(5)7-4(2)6/h1-2H3
5,1052,EI,513-36-0,1,"Propane, 1-chloro-2-methyl-",Isolated,NIST,0.0,"InChI=1S/C4H9Cl/c1-4(2)3-5/h4H,3H2,1-2H3",None,...,CCMSLIB00000980836,215720.0,UPDATE-SINGLE-ANNOTATED-BRONZE,,658.472952,600.240000,0.097016,1788.0,NaN,"1052_InChI=1S/C4H9Cl/c1-4(2)3-5/h4H,3H2,1-2H3"


In [7]:
#keep only unique inchis per feature and select inchi with highest cosine score
#inchis = inchis.groupby('SCAN_Mol', group_keys=False, as_index=False).apply(lambda x: x.loc[x.Cosine.idxmax()])

In [8]:
#select only inchis with a cosine score above 0.9
#inchis.loc[inchis['Cosine'] >= 0.9].shape

In [9]:
matches = [inchis]

In [10]:
out = unique_inchis(matches)
df = out['df']

In [11]:
df.head()

,INCHI
0,"InChI=1S/C7H14/c1-4-5-6-7(2)3/h2,4-6H2,1,3H3"
1,InChI=1S/C13H20O2/c1-9(14)15-7-6-10-4-5-11-8-1...
2,"InChI=1S/C8H16O/c1-6-3-4-8(9)5-7(6)2/h6-9H,3-5..."
3,InChI=1S/C4H4N2/c1-2-6-4-3-5-1/h1-4H
4,"InChI=1S/C15H24O/c1-10-5-6-13-15(4,16-13)8-7-1..."


In [12]:
df.to_csv("INCHIS.csv") # escapechar='&'

In [13]:
! molconvert -g inchi:AuxNone,key,SAbs INCHIS.csv{csv:strucINCHI} -o InchiKeys.txt

Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
INCHIS.csv: error in molecule 114: Cannot recognize format by any of the su

Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo
Omitted undefined stereo


Remark: MOSFIJXAXDLOML-UHFFFAOYSA-N == empty InchiKey

In [14]:
ikeys = pd.read_csv("InchiKeys.txt",  sep='\t',header = None)

In [15]:
ikeys.columns = ["InChIKey"]

In [16]:
ikeys.head()

,InChIKey
0,"InChI=1S/C7H14/c1-4-5-6-7(2)3/h2,4-6H2,1,3H3"
1,InChIKey=IRUDSQHLKGNCGF-UHFFFAOYSA-N
2,InChI=1S/C13H20O2/c1-9(14)15-7-6-10-4-5-11-8-1...
3,InChIKey=AWNOGHRWORTNEI-UHFFFAOYSA-N
4,"InChI=1S/C8H16O/c1-6-3-4-8(9)5-7(6)2/h6-9H,3-5..."


In [17]:
ikeys = pd.DataFrame({'InChI':ikeys['InChIKey'].iloc[::2].values, 'InChIKey':ikeys['InChIKey'].iloc[1::2].values})

In [18]:
ikeys.head()

,InChI,InChIKey
0,"InChI=1S/C7H14/c1-4-5-6-7(2)3/h2,4-6H2,1,3H3",InChIKey=IRUDSQHLKGNCGF-UHFFFAOYSA-N
1,InChI=1S/C13H20O2/c1-9(14)15-7-6-10-4-5-11-8-1...,InChIKey=AWNOGHRWORTNEI-UHFFFAOYSA-N
2,"InChI=1S/C8H16O/c1-6-3-4-8(9)5-7(6)2/h6-9H,3-5...",InChIKey=ZBAXJUPCYVIBSP-UHFFFAOYSA-N
3,InChI=1S/C4H4N2/c1-2-6-4-3-5-1/h1-4H,InChIKey=KYQCOXFCLRTKLS-UHFFFAOYSA-N
4,"InChI=1S/C15H24O/c1-10-5-6-13-15(4,16-13)8-7-1...",InChIKey=NVEQFIOZRFFVFW-UHFFFAOYSA-N


In [19]:
ikeys.InChI = ikeys.InChI.str.replace('=1/', '=1S/')    

In [20]:
ikeys.shape

(1212, 2)

In [21]:
out['df']["inchikey"] = ikeys[['InChIKey']]
inchi_dic = make_inchidic_INCHIS(out)

In [22]:
ikeys[['InChIKey']].to_csv("InchiKeys_ClassyFire.txt", quoting=csv.QUOTE_NONE, escapechar='&')

In [23]:
get_classifications("InchiKeys_ClassyFire.txt")

1000


## create component indexes per network graph

In [25]:
g = read_graphml('GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'MOLECULAR' in x][0])

show all node attributes

In [26]:
list(g.nodes(data=True))

[('1200',
  {'precursor mass': 0.0,
   'RTMean': 0.0,
   'Instrument': 'EI',
   'DefaultGroups': 'N/A',
   'Library_Class': '1',
   'RTConsensus': 0.0,
   'G6': 0.0,
   'G5': 0.0,
   'G4': 0.0,
   'G3': 0.0,
   'G2': 0.0,
   'G1': 0.0,
   'SpectrumFile': 'merged.mgf',
   'tags': ' ',
   'charge': 0,
   'IonMode': 'EI',
   'UniqueFileSources': 'N/A',
   'sum(precursor intensity)': 0.0,
   'PI': 'WILEY',
   'SpectrumID': 'CCMSLIB00002775789',
   'Ion_Source': 'EI',
   'MassDiff': '184.096',
   'Smiles': 'N/A',
   'MQScore': '0.827805',
   'AllGroups': 'N/A',
   'MZErrorPPM': '1e+06',
   'cluster index': 1200,
   'Compound_Name': '3,5-Dimethyl-2,6-bis(methylimino)-3-oxo-tetrahydro-3,5-diazapyran',
   'parent mass': 0.0,
   'Compound_Source': 'Isolated',
   'Adduct': 'EI',
   'SharedPeaks': '14',
   'number of spectra': 0,
   'Data_Collector': 'NIST',
   'INCHI': 'N/A',
   'GNPSLibraryURL': 'http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspectrum.jsp?SpectrumID=CCMSLIB00002775789'}),
 ('344',


convert node attributes of graph to dataframe

In [27]:
net = pd.DataFrame(list(g.nodes(data=True)), columns=['cluster index', 'dics'])

In [28]:
net = net['dics'].apply(pd.Series)

In [29]:
net.head()

,precursor mass,RTMean,Instrument,DefaultGroups,Library_Class,RTConsensus,G6,G5,G4,G3,...,cluster index,Compound_Name,parent mass,Compound_Source,Adduct,SharedPeaks,number of spectra,Data_Collector,INCHI,GNPSLibraryURL
0,0.0,0.0,EI,N/A,1,0.0,0.0,0.0,0.0,0.0,...,1200,"3,5-Dimethyl-2,6-bis(methylimino)-3-oxo-tetrah...",0.0,Isolated,EI,14,0,NIST,N/A,http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...
1,0.0,0.0,EI,N/A,3,0.0,0.0,0.0,0.0,0.0,...,344,"1,3-Dioxolane, 2-hexyl-",0.0,Isolated,EI,10,0,NIST,InChI=1S/C9H18O2/c1-2-3-4-5-6-9-10-7-8-11-9/h9...,http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...
2,0.0,0.0,EI,N/A,1,0.0,0.0,0.0,0.0,0.0,...,345,"Ethanedioic acid, diethyl ester",0.0,Isolated,EI,17,0,NIST,"InChI=1S/C6H10O4/c1-3-9-5(7)6(8)10-4-2/h3-4H2,...",http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...
3,0.0,0.0,EI,N/A,3,0.0,0.0,0.0,0.0,0.0,...,346,Propanoic acid,0.0,Isolated,EI,31,0,NIST,"InChI=1S/C3H6O2/c1-2-3(4)5/h2H2,1H3,(H,4,5)",http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...
4,0.0,0.0,EI,N/A,1,0.0,0.0,0.0,0.0,0.0,...,347,"Hexahydro[1,4]dioxino[2,3-b][1,4]dioxine",0.0,Isolated,EI,6,0,NIST,InChI=1S/C6H10O4/c1-2-8-6-5(7-1)9-3-4-10-6/h5-...,http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...


find all connected components and create component index per graph

In [30]:
connected_components = nx.connected_components(g)

# Find the component id of the nodes
node2id = {}
for cid, component in enumerate(connected_components):
    for node in component:
        node2id[node] = cid

In [31]:
len(node2id)

1596

In [32]:
net.shape

(1596, 36)

In [33]:
net['componentindex'] = net['cluster index']
net['componentindex'] = net['componentindex'].astype(str)
net['componentindex'] = net.componentindex.map(node2id)

In [34]:
net.head()

,precursor mass,RTMean,Instrument,DefaultGroups,Library_Class,RTConsensus,G6,G5,G4,G3,...,Compound_Name,parent mass,Compound_Source,Adduct,SharedPeaks,number of spectra,Data_Collector,INCHI,GNPSLibraryURL,componentindex
0,0.0,0.0,EI,N/A,1,0.0,0.0,0.0,0.0,0.0,...,"3,5-Dimethyl-2,6-bis(methylimino)-3-oxo-tetrah...",0.0,Isolated,EI,14,0,NIST,N/A,http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...,0
1,0.0,0.0,EI,N/A,3,0.0,0.0,0.0,0.0,0.0,...,"1,3-Dioxolane, 2-hexyl-",0.0,Isolated,EI,10,0,NIST,InChI=1S/C9H18O2/c1-2-3-4-5-6-9-10-7-8-11-9/h9...,http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...,1
2,0.0,0.0,EI,N/A,1,0.0,0.0,0.0,0.0,0.0,...,"Ethanedioic acid, diethyl ester",0.0,Isolated,EI,17,0,NIST,"InChI=1S/C6H10O4/c1-3-9-5(7)6(8)10-4-2/h3-4H2,...",http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...,0
3,0.0,0.0,EI,N/A,3,0.0,0.0,0.0,0.0,0.0,...,Propanoic acid,0.0,Isolated,EI,31,0,NIST,"InChI=1S/C3H6O2/c1-2-3(4)5/h2H2,1H3,(H,4,5)",http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...,1
4,0.0,0.0,EI,N/A,1,0.0,0.0,0.0,0.0,0.0,...,"Hexahydro[1,4]dioxino[2,3-b][1,4]dioxine",0.0,Isolated,EI,6,0,NIST,InChI=1S/C6H10O4/c1-2-8-6-5(7-1)9-3-4-10-6/h5-...,http://gnps.ucsd.edu/ProteoSAFe/gnpslibraryspe...,1


In [35]:
net[['cluster index','componentindex']].to_csv("Nodes_withComponentIndex.csv",sep = '\t', index=False)

In [36]:
netfile = "Nodes_withComponentIndex.csv"

In [37]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [38]:
final.head()

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
0,1200,0.0,24,Organic compounds,0.708333,Organic oxygen compounds,0.326389,Organooxygen compounds,0.326389,Carbonyl compounds,0.326389,Ketones,0.211806,Aliphatic acyclic compounds,0.666667
1,345,0.0,24,Organic compounds,0.708333,Organic oxygen compounds,0.326389,Organooxygen compounds,0.326389,Carbonyl compounds,0.326389,Ketones,0.211806,Aliphatic acyclic compounds,0.666667
2,1143,0.0,24,Organic compounds,0.708333,Organic oxygen compounds,0.326389,Organooxygen compounds,0.326389,Carbonyl compounds,0.326389,Ketones,0.211806,Aliphatic acyclic compounds,0.666667
3,54,0.0,24,Organic compounds,0.708333,Organic oxygen compounds,0.326389,Organooxygen compounds,0.326389,Carbonyl compounds,0.326389,Ketones,0.211806,Aliphatic acyclic compounds,0.666667
4,1708,0.0,24,Organic compounds,0.708333,Organic oxygen compounds,0.326389,Organooxygen compounds,0.326389,Carbonyl compounds,0.326389,Ketones,0.211806,Aliphatic acyclic compounds,0.666667


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [39]:
final.to_csv("ClassyFireResults_Network_GCMS.txt", sep = '\t', index = False)

how many InChIKeys could not be classified? (percentage of total submitted InChIKeys)

In [40]:
len(set(list(ikeys.InChIKey)) - set(list(df.inchikey)))/len(set(list(ikeys.InChIKey)))

0.2953795379537954

which InChIKeys could not be classified?

In [41]:
set(list(ikeys.InChIKey)) - set(list(df.inchikey))

{'InChIKey=AAZWWRDLCYIVAB-LUAWRHEFSA-N',
 'InChIKey=ABZZOPIABWYXSN-UHFFFAOYSA-N',
 'InChIKey=ADQJFBQXLAAVQA-UHFFFAOYSA-N',
 'InChIKey=AEXMKKGTQYQZCS-UHFFFAOYSA-N',
 'InChIKey=AFKUSTCGONJZHE-UHFFFAOYSA-N',
 'InChIKey=AJWBFJHTFGRNDG-UHFFFAOYSA-N',
 'InChIKey=AKHMHYGTZRJYBS-UHFFFAOYSA-N',
 'InChIKey=AMBKPYJJYUKNFI-UHFFFAOYSA-N',
 'InChIKey=AMXPURQVAMENCC-PSASIEDQSA-N',
 'InChIKey=AOKRXIIIYJGNNU-UHFFFAOYSA-N',
 'InChIKey=AORMDLNPRGXHHL-UHFFFAOYSA-N',
 'InChIKey=APPOKADJQUIAHP-GLIMQPGKSA-N',
 'InChIKey=ASWTVWBPXCLXKJ-UHFFFAOYSA-N',
 'InChIKey=AUBDSFLQOBEOPX-UHFFFAOYSA-N',
 'InChIKey=AUJLDZJNMXNESO-UHFFFAOYSA-N',
 'InChIKey=AWGBWLXGUPTXHF-UHFFFAOYSA-N',
 'InChIKey=BAGSBIKCNLYDJO-UHFFFAOYSA-N',
 'InChIKey=BAMRUWAHGSFKKX-UHFFFAOYSA-N',
 'InChIKey=BAUZLFKYYIVGPM-UHFFFAOYSA-N',
 'InChIKey=BAYAKMPRFGNNFW-UHFFFAOYSA-N',
 'InChIKey=BEFDCLMNVWHSGT-UHFFFAOYSA-N',
 'InChIKey=BEQGRRJLJLVQAQ-GQCTYLIASA-N',
 'InChIKey=BFDNZQUBFCYTIC-UHFFFAOYSA-N',
 'InChIKey=BGXXXYLRPIRDHJ-UHFFFAOYSA-N',
 'InChIKey=BHELZ

create graphML file

In [42]:
if any("FEATURE" in s for s in os.listdir('GNPS_output_graphML/')):
    graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'FEATURE' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
elif any("MOLECULAR" in s for s in os.listdir('GNPS_output_graphML/')):
    graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'MOLECULAR' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
elif any("METABOLOMICS" in s for s in os.listdir('GNPS_output_graphML/')):
    graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'METABOLOMICS' in x][0]
    graphML = read_graphml(graphMLfile)
    graphML_classy = make_classyfire_graphml(graphML,final)
    nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)
else:
    print('There is no graphML file for this GNPS molecular network job')